In [39]:
import pandas as pd
from sklearn.metrics import classification_report, accuracy_score

In [41]:
df=pd.read_csv("C:\\Users\\suhan\\Downloads\\customer_churn_dataset-testing-master.csv\\customer_churn_dataset-testing-master.csv")

In [43]:
df.head()

,CustomerID,Age,Gender,Tenure,Usage Frequency,Support Calls,Payment Delay,Subscription Type,Contract Length,Total Spend,Last Interaction,Churn
0,1,22,Female,25,14,4,27,Basic,Monthly,598,9,1
1,2,41,Female,28,28,7,13,Standard,Monthly,584,20,0
2,3,47,Male,27,10,2,29,Premium,Annual,757,21,0
3,4,35,Male,9,12,5,17,Premium,Quarterly,232,18,0
4,5,53,Female,58,24,9,2,Standard,Annual,533,18,0


In [45]:
X = df.drop('Churn', axis=1)
y = df['Churn']

# encode 

In [96]:
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer

In [47]:
onehot_cols = ['Gender']
label_cols = ['Subscription Type','Contract Length']

In [49]:
for col in label_cols:
    le = LabelEncoder()
    X[col] = le.fit_transform(X[col])

In [51]:
ct = ColumnTransformer(
    transformers=[
        ('onehot', OneHotEncoder(drop='first'), onehot_cols)  # drop='first' to avoid dummy variable trap
    ], remainder='passthrough'  # keep other columns as is
)


In [53]:
X = X.drop('CustomerID', axis=1)

In [55]:
X_encoded = ct.fit_transform(X)

# train and test split

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

In [57]:
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.2, random_state=42)

In [61]:
model = LogisticRegression(max_iter=2000)
model.fit(X_train, y_train)


LogisticRegression(max_iter=2000)

# predict

In [62]:
y_pred = model.predict(X_test)

# metrics

In [65]:
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

Accuracy: 0.8306796116504854
              precision    recall  f1-score   support

           0       0.85      0.83      0.84      6793
           1       0.81      0.83      0.82      6082

    accuracy                           0.83     12875
   macro avg       0.83      0.83      0.83     12875
weighted avg       0.83      0.83      0.83     12875



# gridsearcv for better accuracy

In [67]:
param_grid = {
    'C': [0.01, 0.1, 1, 10, 100],      # regularization strength
    'penalty': ['l1', 'l2']             # regularization type
}

In [69]:
from sklearn.model_selection import GridSearchCV

In [71]:
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, 
                           cv=5, scoring='accuracy', n_jobs=-1)

In [73]:
grid_search.fit(X_train, y_train)

C:\Users\suhan\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:540: FitFailedWarning: 
25 fits failed out of a total of 50.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
25 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\suhan\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\suhan\anaconda3\Lib\site-packages\sklearn\base.py", line 1473, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\suhan\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py", line 1194, in fit
    solver = _

GridSearchCV(cv=5, estimator=LogisticRegression(max_iter=2000), n_jobs=-1,
             param_grid={'C': [0.01, 0.1, 1, 10, 100], 'penalty': ['l1', 'l2']},
             scoring='accuracy')

In [74]:
print("Best parameters found:", grid_search.best_params_)
print(grid_search.best_score_)

Best parameters found: {'C': 0.1, 'penalty': 'l2'}
0.8239384575936771


In [75]:
best_model = grid_search.best_estimator_

In [76]:
y_pred = best_model.predict(X_test)

In [77]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))
print(accuracy_score(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.85      0.83      0.84      6793
           1       0.81      0.83      0.82      6082

    accuracy                           0.83     12875
   macro avg       0.83      0.83      0.83     12875
weighted avg       0.83      0.83      0.83     12875

0.8305242718446602


# pickle

In [85]:
import pickle

In [87]:
pickle.dump(model,open('model.pkl','wb'))

In [91]:
model1=pickle.load(open('model.pkl','rb'))

In [93]:
model1.predict(X_train)

array([0, 0, 0, ..., 1, 0, 0], dtype=int64)